# Sentinel-2 Sunglint Finder with Earth Engine Python API

This notebook shows how to find Sentinel-2 scenes over sunglint geometry using the Earth Engine Python API.  
Adjust the `bbox`, `start_date`, `end_date`, and other parameters as needed.


In [ ]:
# 1. Install and import Earth Engine Python API
# Uncomment the following line if running in Colab or a new environment:
# !pip install earthengine-api
%load_ext autoreload
%autoreload 2

import ee
import geopandas as gpd
import geemap
from sunglint_raster import add_sgi_b8a_b3, add_abascal_sun_glint_bands, print_s2_metadata_angles, expanded_into_mosaic, add_cloud_probability, plot_mosaic_scatter ,mosaic_scatter, add_sgi_b12_b3
import pandas as pd
from shapely.geometry import Polygon, Point

# ee.Authenticate()
ee.Initialize()
print('Earth Engine initialized.')


### Load Infrastructure in the Gulf of Mexico

In [ ]:

infra = pd.read_csv(r"C:\Users\ebeva\SkyTruth\methane\nonoise_SAR_fixed_infrastructure.csv")

# 1. Define the Gulf of Mexico bounding box
min_lon, min_lat = -98.0, 24.0
max_lon, max_lat = -80.5, 31.0

# Define the polygon as a shapely Polygon (longitude, latitude order)
gulf_poly = Polygon([
    (-90.384631, 23.859054),
    (-90.384631, 28.645609),
    (-85.68112, 28.645609),
    (-85.68112, 23.859054),
    (-90.384631, 23.859054)
])

# Create GeoDataFrame from infra DataFrame
gdf = gpd.GeoDataFrame(
    infra,
    geometry=[Point(xy) for xy in zip(infra['lon'], infra['lat'])],
    crs="EPSG:4326"
)

# Filter gdf by whether points are within the polygon
gdf_gulf = gdf[gdf.geometry.within(gulf_poly)]
gdf_gulf.plot()

### Find a specific infrastructure in the gulf

In [ ]:

# infra.sample(1)
row = gdf_gulf.sample(1).iloc[0]
stationary_infra = ee.Geometry.Point([row['lon'], row['lat']])
Map = geemap.Map()
Map.addLayer(stationary_infra)
Map.centerObject(stationary_infra, 6)
Map

### Pull Imagery and display

In [ ]:
def get_s2_collection(start_date, end_date, stationary_infra, cloud_percent=20):
    return (
        ee.ImageCollection('COPERNICUS/S2_SR')
        .filterDate(start_date, end_date)
        .filterBounds(stationary_infra)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percent))
    )

s2 = get_s2_collection('2018-01-01', '2025-12-01', stationary_infra=stationary_infra)

In [ ]:
sorted_s2 = s2.sort('MEAN_SOLAR_ZENITH_ANGLE').toList(10)
idx = 1
glint_example = ee.Image(sorted_s2.get(idx))

In [ ]:
print_s2_metadata_angles(glint_example)

In [ ]:

# sgi_b8a_b3 = add_sgi_b8a_b3(glint_example).select("sgi_b8a_b3")
# abascal_glint_score = add_abascal_sun_glint_bands(glint_example).select("abascal_glint_score")
abascal_glint_score = expanded_into_mosaic(glint_example, band_operation_fn=add_abascal_sun_glint_bands, band_name="abascal_glint_score")
sgi_b8a_b3 = expanded_into_mosaic(glint_example,add_sgi_b8a_b3,"sgi_b8a_b3")
sgi_b12_b3 = expanded_into_mosaic(glint_example,add_sgi_b12_b3,"sgi_b12_b3")
glint_example_cloud = expanded_into_mosaic(glint_example, add_cloud_probability, "cloud_probability")
glint_example_mosiac = expanded_into_mosaic(glint_example)

In [ ]:
# Visualize RGB - .visualize(bands=['B4', 'B3', 'B2'], min=0, max=4000)
# Visualize single band {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'red']}
Map = geemap.Map()

Map.addLayer(glint_example_mosiac.visualize(bands=['B12'], min=0, max=3000),{}, 'S2 b12', False)
# Map.addLayer(glint_example_cloud, {'min': 0, 'max': 100, 'palette': ['blue', 'white', 'red']}, "cloud_probability", False)
# Map.addLayer(abascal_glint_score, {'min': 0, 'max': 20, 'palette': ['blue', 'white', 'red']}, "abascal_glint", False)


Map.addLayer(sgi_b8a_b3, {'min': -.5, 'max': .5, 'palette': ['blue', 'white', 'red']}, "sgi_b8a_b3")
Map.addLayer(sgi_b12_b3, {'min': -.5, 'max': .5, 'palette': ['blue', 'white', 'red']}, "sgi_b12_b3")

#Zhang et al. SGI < -0.5 indicates no glint, i,e glint is -0.5 and above
Map.addLayer(sgi_b12_b3.gt(-0.1), {'min': -.5, 'max': .5, 'palette': ['blue', 'white', 'red']}, "sgi_mask")

Map.addLayer(sgi_b12_b3.subtract(sgi_b8a_b3), {'min': -.05, 'max': .05, 'palette': ['blue', 'white', 'red']}, "subtract", False)
Map.addLayer(glint_example_mosiac.visualize(bands=['B4', 'B3', 'B2'], min=0, max=4000),{}, 'Color S2')
Map.addLayer(stationary_infra,{'color': 'red'}, 'Stationary')

Map.centerObject(stationary_infra, 9)
Map

In [ ]:
# roi_feature = Map.draw_last_feature
# if roi_feature is not None:
#     scatter = mosaic_scatter(glint_example_mosiac.select("B4"), sgi_b8a_b3, roi_feature.geometry()) #bx_name="B13", by_name="SGI")
